In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle
from torch.utils.data import TensorDataset, DataLoader


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
BATCH_SIZE = 1024

In [4]:
# Load dataset

np.random.seed(32)

with open('circleDatasetAug.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

dataset_size = len(dataset)

def split_indices(n, val_pct):
    n_val = int(val_pct*n)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]

train_idxs, val_idxs = split_indices(dataset_size, 0.2)

train_data = [dataset[i] for i in train_idxs]
val_data = [dataset[i] for i in val_idxs]

train_data = torch.Tensor(train_data)
val_data = torch.Tensor(train_data)

train_dataset = TensorDataset(train_data)
val_dataset = TensorDataset(val_data)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

C:\Users\jorge\AppData\Local\Temp\ipykernel_19296\2881436632.py:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:278.)
  train_data = torch.Tensor(train_data)


In [5]:
class ColdDiff:
    def __init__(self, steps=300, size=1000, loss_type='L2'):
        self.steps = steps
        self.size = size
        self.loss_type = loss_type
        
        
    def displacement(self, step, max_steps, amplitude=0.1):
        return amplitude * np.sin(2*np.pi*step/max_steps)
    
    def sample_timesteps(self, n):
        return torch.randint(low=1, high=self.steps, size=(n,))
    
    def degradation(self, points, step, points_per_step, max_steps, radius=1.0, amplitude=0.1, batch_size=BATCH_SIZE):
        new_points = points.clone()
        batch_size, dim, num_points = points.shape
        norms = torch.norm(new_points, dim=1, keepdim=True)
        directions = new_points/(norms+1e-8)
        displacement = self.displacement(step, max_steps)
                
        for b in range(batch_size):
            idxs = np.random.choice(num_points, points_per_step, replace=False)
            new_points[b, :, idxs] += directions[b, :, idxs]*displacement

        return new_points
    
    def degradation_throughTime(self, points, t, points_per_step, max_steps, radius=1.0, amplitude=0.1):
        for b in tqdm(range(points.shape[0])):
            steps_to_apply = t[b].item()
            for i in tqdm(range(steps_to_apply)):
                prev = points
                points = self.degradation(points, i, points_per_step, max_steps)
        return points, prev
        
#     def sample(self, model, x_t):
#         # Algo 2
#         model.eval()
#         x_sPrev = x_t
#         for s in tqdm(reversed(range(1, self.steps)), position=0):
#             x_0 = model(x_sPrev, s) #Unet
#             D_s, D_sPrev = self.degradation_throughTime(x_0, s, points_per_step=10, max_steps=self.max_steps)
#             x_sPrev = x_0 - D_s + D_sPrev 
        
#         return x_0

    def sample(self, model, img, t):
        model.eval()
        x_sPrev = img
        while(t):
            step = torch.full((batch_size,), t-1, dtype=torch.long).cuda()
            x_0 = model(x_sPrev, step)
            for i in range(t):
                with torch.no_grad():
                    D_s, D_sPrev = self.degradation_throughTime(x_0, )

In [6]:
from torch import nn

class UNet(nn.Module):
    def __init__(self, c_in = 2, c_out=1, time_dim=256):
        super().__init__()
        self.device = device
        self.time_dim = time_dim
        self.inc = DoubleConv(c_in, 64)
        self.down1 = Down(64, 128)
        self.sa1 = SelfAttention(128, 1000//2)
        self.down2 = Down(128, 256)
        self.sa2 = SelfAttention(256, 1000//4)
        self.down3 = Down(256, 256)
        self.sa3 = SelfAttention(256, 1000//8)
        
        self.bot1 = DoubleConv(256, 512)
        self.bot2 = DoubleConv(512, 512)
        self.bot3 = DoubleConv(512, 256)
        
        self.up1 = Up(512, 128)
        self.sa4 = SelfAttention(128, 1000//4)
        self.up2 = Up(256, 64)
        self.sa5 = SelfAttention(64, 1000//2)
        self.up3 = Up(128, 64)
        self.sa6 = SelfAttention(64,1000)
        self.outc = nn.Conv1d(64, c_out, kernel_size=1)
        
    # Sinosoidal encoding - further read
    def pos_encoding(self, t, channels):
        inv_freq = 1.0 / (10000**(torch.arange(0, channels, 2, device=self.device).float() / channels))
        
        pos_enc_a = torch.sin(t.repeat(1, channels // 2)*inv_freq)
        pos_enc_b = torch.cos(t.repeat(1, channels//2)*inv_freq)
        pos_enc = torch.cat([pos_enc_a, pos_enc_b], dim=-1)
        return pos_enc
    
    def forward(self, x, t):
        t = t.unsqueeze(-1).type(torch.float)
        t = self.pos_encoding(t, self.time_dim)
        
        # Encoder
        x1 = self.inc(x)
        x2 = self.down1(x1, t)
        x2 = self.sa1(x2)
        x3 = self.down2(x2, t)
        x3 = self.sa2(x3)
        x4 = self.down3(x3, t)
        x4 = self.sa3(x4)
        
        # Bottleneck
        x4 = self.bot1(x4)
        x4 = self.bot2(x4)
        x4 = self.bot3(x4)
        
        # Decoder
        #print(x4.size())
        #print(x3.size())
        x = self.up1(x4, x3, t)
        x = self.sa4(x)
        x = self.up2(x, x2, t)
        x = self.sa5(x)
        x = self.up3(x, x1, t)
        x = self.sa6(x)
        return self.outc(x)  

In [7]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None, residual=False):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.conv1 = nn.Conv1d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False)
        self.norm1 = nn.BatchNorm1d(mid_channels)
        self.act = nn.ReLU() ## Try Relu, leakyReLU
        self.conv2 = nn.Conv1d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.norm2 = nn.BatchNorm1d(out_channels)
        self.residual = residual
        
    def forward(self, x):
        x2 = self.conv1(x)
        x2 = self.norm1(x2)
        x2 = self.act(x2)
        x2 = self.conv2(x2)
        x2 = self.norm2(x2)
        if self.residual:
            return self.act(x+x2)
        else:
            return x2

In [8]:
class Down(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()
        self.maxPool = nn.MaxPool1d(2)
        self.doubleConv1 = DoubleConv(in_channels, in_channels, residual=True)
        self.doubleConv2 = DoubleConv(in_channels, out_channels)
        
        self.act = nn.ReLU()
        self.linear = nn.Linear(emb_dim, out_channels)
        
    def forward(self, x, t):
        x = self.maxPool(x)
        x = self.doubleConv1(x)
        x = self.doubleConv2(x)
        #print(x.size())
        emb = self.act(t)
        emb = self.linear(emb)[:, :, None].repeat(1, 1, x.shape[-1])
        #print(emb.size())
        
        return x+emb 

In [9]:
class Up(nn.Module):
    def __init__(self, in_channels, out_channels, emb_dim=256):
        super().__init__()
        
        #self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        self.up = nn.ConvTranspose1d(in_channels // 2, in_channels // 2, kernel_size=2, stride=2)
        self.doubleConv1 = DoubleConv(in_channels, in_channels, residual=True)
        self.doubleConv2 = DoubleConv(in_channels, out_channels, in_channels//2)
        self.act = nn.ReLU()
        self.linear = nn.Linear(emb_dim, out_channels)
        
    def forward(self, x, skip_x, t):
        #print(x.size())
        x = self.up(x)
        #print(x.size())
        if x.shape[-2:] != skip_x.shape[-2:]:
            x = nn.functional.interpolate(x, size=skip_x.shape[-2:], mode='bilinear', align_corners=True)
            #print(x.size())
        x = torch.cat([skip_x, x], dim=1)
        x = self.doubleConv1(x)
        x = self.doubleConv2(x)
        emb = self.act(t)
        emb = self.linear(emb)[:, :, None].repeat(1,1, x.shape[-1])
        return x+emb

In [10]:
class SelfAttention(nn.Module):
    def __init__(self, channels, size):
        super(SelfAttention, self).__init__()
        self.channels = channels
        self.size = size
        self.mha = nn.MultiheadAttention(channels, 4, batch_first=True)
        self.ln = nn.LayerNorm([channels])
        self.linear = nn.Linear(channels, channels)
        self.act = nn.ReLU()
    
    def forward(self, x):
        b, c, p = x.shape
        #x = x.view(b, c, h*w).permute(0,2,1)
        x = x.permute(0,2,1)
        x_ln = self.ln(x)
        attention_value, _ = self.mha(x_ln, x_ln, x_ln)
        attention_value = attention_value + x
        x = self.ln(attention_value)
        x = self.linear(x)
        x = self.act(x)
        x = self.linear(x)
        attention_value = x + attention_value
        
        return attention_value.permute(0, 2, 1)#.view(b, c, h, w)

In [11]:
def visualize_2D(x,y):
    fig = plt.figure()
    plt.scatter(x, y, s=5)
    plt.axis('equal')
    plt.title('Training Data: Points on a Circle')
    plt.show()

In [12]:
EPOCHS = 50
image_size = 1000
learning_rate = 3e-4

# Import dataset

model = UNet().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss() 
# loss_fn = nn.L1Loss()
# loss_fn = nn.SmoothL1Loss() # L1 Huber loss
# loss_fn = nn.CosineSimilarity(dim=1)
diffusion = ColdDiff()
length = len(train_data)
val_losses = []
train_losses = []

for epoch in tqdm(range(EPOCHS)):
    train_loss = 0
    for x in train_loader:
        optimizer.zero_grad()
        x = x[0]
        x = x.to(device)
        t = diffusion.sample_timesteps(x.shape[0]).to(device)
        x_t, _ = diffusion.degradation_throughTime(x, t, points_per_step=10, max_steps=diffusion.steps)
        print("finish sampling")
        
        predicted_x0 = model(x_t, t)
        print("finish training")
        
        loss = loss_fn(x, predicted_x0)
        # loss = 1 - loss_fn(noise, predicted_noise).mean() # for cos sim
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    val_loss = 0
    with torch.no_grad():
        model.eval()
        for x in val_loader:
            x = x[0]
            x = x.to(device)
            t = diffusion.sample_timesteps(x.shape[0]).to(device)
            x_t, _ = diffusion.degradation_throughTime(x, t, points_per_step=10)
        
            predicted_x0 = model(x_t, t)
            batch_val_loss = loss_fn(x, predicted_x0)
            val_loss += batch_val_loss.item()
            
    sampled_images = diffusion.sample(model, 5)
    for i in sampled_images:
        x, y = i.detach().cpu().numpy()
        visualize_2D(x, y)
    
    avg_train_loss = train_loss/len(train_loader)
    avg_val_loss = val_loss/len(val_loader)
    print(f"Epoch [{epoch+1}/100] | Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    

  0%|                                                                                         | 0/1024 [00:00<?, ?it/s]

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

  3%|██▊                                                                                | 1/30 [00:00<00:12,  2.40it/s]

  7%|█████▌                                                                             | 2/30 [00:00<00:11,  2.51it/s]

 10%|████████▎                                                                          | 3/30 [00:01<00:10,  2.61it/s]

 13%|███████████                                                                        | 4/30 [00:01<00:09,  2.62it/s]

 17%|█████████████▊                                                                     | 5/30 [00:01<00:09,  2.68it/s]

 20%|████████████████▌                                                                  | 6/30 [00:02<00:08,  2.67it/s]

 23%|███████████████████▎       

 14%|███████████▋                                                                     | 30/209 [00:14<01:24,  2.12it/s]

 15%|████████████                                                                     | 31/209 [00:14<01:23,  2.13it/s]

 15%|████████████▍                                                                    | 32/209 [00:15<01:22,  2.15it/s]

 16%|████████████▊                                                                    | 33/209 [00:15<01:21,  2.17it/s]

 16%|█████████████▏                                                                   | 34/209 [00:16<01:20,  2.17it/s]

 17%|█████████████▌                                                                   | 35/209 [00:16<01:20,  2.16it/s]

 17%|█████████████▉                                                                   | 36/209 [00:16<01:20,  2.16it/s]

 18%|██████████████▎                                                                  | 37/209 [00:17<01:19,  2.15it/s]

 18%|██████████████▋            

 45%|████████████████████████████████████▍                                            | 94/209 [00:44<00:53,  2.17it/s]

 45%|████████████████████████████████████▊                                            | 95/209 [00:44<00:52,  2.16it/s]

 46%|█████████████████████████████████████▏                                           | 96/209 [00:45<00:52,  2.13it/s]

 46%|█████████████████████████████████████▌                                           | 97/209 [00:45<00:52,  2.14it/s]

 47%|█████████████████████████████████████▉                                           | 98/209 [00:45<00:52,  2.13it/s]

 47%|██████████████████████████████████████▎                                          | 99/209 [00:46<00:51,  2.13it/s]

 48%|██████████████████████████████████████▎                                         | 100/209 [00:46<00:50,  2.14it/s]

 48%|██████████████████████████████████████▋                                         | 101/209 [00:47<00:50,  2.15it/s]

 49%|███████████████████████████

 76%|████████████████████████████████████████████████████████████▍                   | 158/209 [01:14<00:23,  2.16it/s]

 76%|████████████████████████████████████████████████████████████▊                   | 159/209 [01:14<00:23,  2.15it/s]

 77%|█████████████████████████████████████████████████████████████▏                  | 160/209 [01:15<00:22,  2.14it/s]

 77%|█████████████████████████████████████████████████████████████▋                  | 161/209 [01:15<00:22,  2.15it/s]

 78%|██████████████████████████████████████████████████████████████                  | 162/209 [01:15<00:21,  2.14it/s]

 78%|██████████████████████████████████████████████████████████████▍                 | 163/209 [01:16<00:21,  2.13it/s]

 78%|██████████████████████████████████████████████████████████████▊                 | 164/209 [01:16<00:21,  2.13it/s]

 79%|███████████████████████████████████████████████████████████████▏                | 165/209 [01:17<00:20,  2.14it/s]

 79%|███████████████████████████

 10%|███████▉                                                                         | 11/113 [00:05<00:47,  2.15it/s]

 11%|████████▌                                                                        | 12/113 [00:05<00:47,  2.13it/s]

 12%|█████████▎                                                                       | 13/113 [00:06<00:46,  2.13it/s]

 12%|██████████                                                                       | 14/113 [00:06<00:46,  2.14it/s]

 13%|██████████▊                                                                      | 15/113 [00:06<00:45,  2.14it/s]

 14%|███████████▍                                                                     | 16/113 [00:07<00:45,  2.12it/s]

 15%|████████████▏                                                                    | 17/113 [00:07<00:45,  2.12it/s]

 16%|████████████▉                                                                    | 18/113 [00:08<00:44,  2.13it/s]

 17%|█████████████▌             

...

 56%|████████████████████████████████████████████▍                                   | 146/263 [01:07<00:54,  2.15it/s]

 56%|████████████████████████████████████████████▋                                   | 147/263 [01:08<00:53,  2.15it/s]

 56%|█████████████████████████████████████████████                                   | 148/263 [01:08<00:53,  2.15it/s]

 57%|█████████████████████████████████████████████▎                                  | 149/263 [01:09<00:53,  2.15it/s]

 57%|█████████████████████████████████████████████▋                                  | 150/263 [01:09<00:52,  2.16it/s]

 57%|█████████████████████████████████████████████▉                                  | 151/263 [01:10<00:52,  2.15it/s]

 58%|██████████████████████████████████████████████▏                                 | 152/263 [01:10<00:51,  2.15it/s]

 58%|██████████████████████████████████████████████▌                                 | 153/263 [01:11<00:51,  2.15it/s]

 59%|███████████████████████████

 80%|███████████████████████████████████████████████████████████████▉                | 210/263 [01:37<00:24,  2.13it/s]

 80%|████████████████████████████████████████████████████████████████▏               | 211/263 [01:38<00:24,  2.13it/s]

 81%|████████████████████████████████████████████████████████████████▍               | 212/263 [01:38<00:23,  2.15it/s]

 81%|████████████████████████████████████████████████████████████████▊               | 213/263 [01:39<00:23,  2.16it/s]

 81%|█████████████████████████████████████████████████████████████████               | 214/263 [01:39<00:22,  2.16it/s]

 82%|█████████████████████████████████████████████████████████████████▍              | 215/263 [01:39<00:22,  2.17it/s]

 82%|█████████████████████████████████████████████████████████████████▋              | 216/263 [01:40<00:21,  2.17it/s]

 83%|██████████████████████████████████████████████████████████████████              | 217/263 [01:40<00:21,  2.16it/s]

 83%|███████████████████████████

 11%|█████████▎                                                                         | 9/80 [00:04<00:33,  2.14it/s]

 12%|██████████▎                                                                       | 10/80 [00:04<00:32,  2.13it/s]

 14%|███████████▎                                                                      | 11/80 [00:05<00:32,  2.13it/s]

 15%|████████████▎                                                                     | 12/80 [00:05<00:31,  2.15it/s]

 16%|█████████████▎                                                                    | 13/80 [00:06<00:31,  2.16it/s]

 18%|██████████████▎                                                                   | 14/80 [00:06<00:30,  2.16it/s]

 19%|███████████████▍                                                                  | 15/80 [00:06<00:30,  2.16it/s]

 20%|████████████████▍                                                                 | 16/80 [00:07<00:29,  2.17it/s]

 21%|█████████████████▍         

 91%|██████████████████████████████████████████████████████████████████████████▊       | 73/80 [00:33<00:03,  2.17it/s]

 92%|███████████████████████████████████████████████████████████████████████████▊      | 74/80 [00:34<00:02,  2.17it/s]

 94%|████████████████████████████████████████████████████████████████████████████▉     | 75/80 [00:34<00:02,  2.17it/s]

 95%|█████████████████████████████████████████████████████████████████████████████▉    | 76/80 [00:35<00:01,  2.16it/s]

 96%|██████████████████████████████████████████████████████████████████████████████▉   | 77/80 [00:35<00:01,  2.14it/s]

 98%|███████████████████████████████████████████████████████████████████████████████▉  | 78/80 [00:36<00:00,  2.14it/s]

 99%|████████████████████████████████████████████████████████████████████████████████▉ | 79/80 [00:36<00:00,  2.14it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:37<00:00,  2.15it/s]

  9%|██████▍                    

 22%|█████████████████▉                                                               | 55/249 [00:25<01:29,  2.16it/s]

 22%|██████████████████▏                                                              | 56/249 [00:26<01:29,  2.16it/s]

 23%|██████████████████▌                                                              | 57/249 [00:26<01:28,  2.16it/s]

 23%|██████████████████▊                                                              | 58/249 [00:26<01:28,  2.16it/s]

 24%|███████████████████▏                                                             | 59/249 [00:27<01:28,  2.16it/s]

In [ ]:
new_points.device